# Домашнее задание к лекции «Проблема качества данных»

In [376]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

Создаю синтетический дата-сет.
Он будет представлять собой медицинские данные, собранные с карточек пациентов. Предположим, что сбор осуществлялся вручную в нескольких больницах города, только для пациентов кардиологии (поэтому ограниченный набор диагнозов).

В датасете есть следующие переменные:
- patient_id - номер пациента, уникальный
- hospital_id - номер больницы, уникальный
- card_number - номер карты пациента, неуникальный, 70% пропусков
- temperature - температура
- pulse - пульс
- diagnosis - диагноз, категориальная переменная
- time_arrival и time_checkout - время поступления в больницу и время выписки
- recovery - прогноз полного выздоровления. Это целевая переменная. Предположим, что пациентов наблюдают втечении жизни, и чем большее время пациент не имеет повторных проблем с сердцем, то выше его recovery. recovery у нас будет завсить от месяца, в котором пациент попал в больницу и от времени, которое он в ней провел (понимаю, что это очень недостоверно, но делать зависимость от пульса и температуры показалось мне слишком простым)

In [377]:
n_samples = 1000

# Создаю id пациента и больницы
patient_id = np.arange(1, 1001)
hospital_id = np.random.choice(5, n_samples) + 50

# Создаю номер карты пациента, искуственно добавляю пропуски

card_number_bul = np.random.choice([True, False], n_samples, p=[0.3, 0.7])
card_number = np.random.choice(np.arange(1001., 3000.), n_samples)

card_number.ravel()[np.random.choice(card_number.size, 700, replace=False)] = np.nan

# Заполняю диагноз
diagnosis = np.random.choice(['infarct', 'insult', 'arrhythmia'], n_samples)

# Заполняю температуру и пульс
temperature = np.random.choice(np.arange(34, 41, 0.1), n_samples)
pulse = np.random.choice(np.arange(50, 300), n_samples)

# Заполняю даты случайными временными рядами.
# Здесь надо сказать, что логичное требование к этим датам: Дата поступления не может быть позже даты выписки. 
# Поэтому я ставлю дату поступления в более раннем диапазоне. Но так как у нас тема о проблемах с данными, оставляю  
# пересечение между диапазонами. Это намеренная ошибка в данных, которую мы обработаем позднее.

def random_date_generator(start_date, range_in_days):
    days_to_add = np.arange(0, range_in_days)
    random_date = np.datetime64(start_date) + np.random.choice(days_to_add)
    return random_date

time_arrival_full = np.random.choice(np.arange(1, 300), n_samples)
time_checkout_full = np.random.choice(np.arange(1, 300), n_samples)
time_arrival = []
time_checkout = []

for i in range(0, 1000):
    time_arrival.append(random_date_generator('2019-01-01', time_arrival_full[i]))
    time_checkout.append(random_date_generator('2019-06-30', time_checkout_full[i]))

# Заполняю прогноз выздоровления
recovery = []

for i in range(0, 1000):
    delta = (time_checkout[i] - time_arrival[i]).item().days # дни, проведённые в больнице
    date = pd.to_datetime(time_arrival[i]) # месяц попадания в больницу
    month = date.month
    recovery.append(delta * month + 333)

In [378]:
# Итоговый датасет выглядит так:

data = pd.DataFrame({'patient_id': patient_id, 'hospital_id': hospital_id, 'card_number': card_number, 'diagnosis': diagnosis,
                    'temperature': temperature , 'pulse': pulse, 'time_arrival': time_arrival, 'time_checkout': time_checkout, 
                     'recovery': recovery})
data.head(10)

,patient_id,hospital_id,card_number,diagnosis,temperature,pulse,time_arrival,time_checkout,recovery
0,1,53,NaN,insult,34.1,182,2019-02-20,2019-09-07,731
1,2,52,NaN,arrhythmia,40.5,281,2019-04-22,2019-10-20,1057
2,3,52,2840.0,infarct,37.7,212,2019-04-08,2019-08-01,793
3,4,54,NaN,insult,40.7,68,2019-07-16,2020-01-27,1698
4,5,50,NaN,arrhythmia,35.8,162,2019-05-16,2019-11-29,1318
5,6,51,NaN,arrhythmia,38.4,115,2019-01-30,2019-10-27,603
6,7,51,2021.0,infarct,37.4,147,2019-02-19,2019-12-28,957
7,8,51,NaN,arrhythmia,34.8,215,2019-01-25,2019-09-03,554
8,9,50,NaN,arrhythmia,34.8,150,2019-04-25,2019-11-09,1125
9,10,50,2445.0,infarct,39.9,51,2019-03-30,2019-09-23,864


In [379]:
# Вспомогательные функции:
    
def get_score(X, y):
    reg = LinearRegression().fit(X, y)
    print('Weights: {}'.format(reg.coef_))
    print('Bias: {}'.format(reg.intercept_))
    pred_values = reg.predict(X)
    print('Error: {}'.format(mean_absolute_error(pred_values, y)))

def get_one_hot(X, cols):
    for each in cols:
        dummies = pd.get_dummies(X[each], prefix=each, drop_first=False)
        X = pd.concat([X, dummies], axis=1)
    return X

Пока единственные переменные, которые мы можем использовать, это температура и пульс. Вот так выглядит результат обучения в этом случае:

In [380]:
get_score(data[['temperature', 'pulse']], data.recovery)

Weights: [8.33620407 0.05839133]
Bias: 421.8885841191647
Error: 217.2789643817714


Для начала небольшая работа с пропусками. Они есть в колонке карточка товара. Причины, почему они могли возникнуть:
- Больница не нумерует карточки
- Больница раньше нумеровала карточки, а потом перестала (или наоборот)
- Человек, собиравший данные, не внёс номер карточки
На мой взгляд такие данные имеют весьма низкую ценность: во-первых их мало, во-вторых здравый смысл подсказывает, что корреляция между номером карточки и прогнозом выздоровления если и существует, то вряд ли будет линейной. В данном случае удаляем весь столбец.

In [381]:
data = data.drop('card_number', axis=1)

data.head()

,patient_id,hospital_id,diagnosis,temperature,pulse,time_arrival,time_checkout,recovery
0,1,53,insult,34.1,182,2019-02-20,2019-09-07,731
1,2,52,arrhythmia,40.5,281,2019-04-22,2019-10-20,1057
2,3,52,infarct,37.7,212,2019-04-08,2019-08-01,793
3,4,54,insult,40.7,68,2019-07-16,2020-01-27,1698
4,5,50,arrhythmia,35.8,162,2019-05-16,2019-11-29,1318


Сделаем ещё одно преобразование. У нас не работает диагноз, но в том виде, в котором он есть, его использовать нельзя. Так как вариантов диагнозов всего 3, используем one hot encoding

In [382]:
data = get_one_hot(data, data[['diagnosis']])

data.head()

,patient_id,hospital_id,diagnosis,temperature,pulse,time_arrival,time_checkout,recovery,diagnosis_arrhythmia,diagnosis_infarct,diagnosis_insult
0,1,53,insult,34.1,182,2019-02-20,2019-09-07,731,0,0,1
1,2,52,arrhythmia,40.5,281,2019-04-22,2019-10-20,1057,1,0,0
2,3,52,infarct,37.7,212,2019-04-08,2019-08-01,793,0,1,0
3,4,54,insult,40.7,68,2019-07-16,2020-01-27,1698,0,0,1
4,5,50,arrhythmia,35.8,162,2019-05-16,2019-11-29,1318,1,0,0


И попробуем обучить модель ещё раз.

In [383]:
get_score(data[['temperature', 'pulse', 'diagnosis_arrhythmia', 'diagnosis_insult', 'diagnosis_infarct']], data.recovery)

Weights: [ 8.41848787  0.05908644  4.28574825  2.47825344 -6.7640017 ]
Bias: 419.12166628484266
Error: 217.15234565429296


Особо лучше не стало, оно и понятно, так как корреляций с диагнозом не закладывалось. Для наглядности удалим эти колонки

In [384]:
data = data.drop(['diagnosis_arrhythmia', 'diagnosis_insult', 'diagnosis_infarct'], axis=1)

data.head()

,patient_id,hospital_id,diagnosis,temperature,pulse,time_arrival,time_checkout,recovery
0,1,53,insult,34.1,182,2019-02-20,2019-09-07,731
1,2,52,arrhythmia,40.5,281,2019-04-22,2019-10-20,1057
2,3,52,infarct,37.7,212,2019-04-08,2019-08-01,793
3,4,54,insult,40.7,68,2019-07-16,2020-01-27,1698
4,5,50,arrhythmia,35.8,162,2019-05-16,2019-11-29,1318


Теперь попробуем добавить новые признаки. Для этого возьмём наши временные ряды: дата поступления и дата выписки.

- Добавим новый признак days_illness - кол-во дней болезни
- Проверим, есть ли ситуации, когда значение этого признака отрицательное.
Если есть, то это ошибка. Мы можем либо удалить строки с ошибкой, либо, например, поменять даты местами (так как данные собирались вручную, возможно это ошибка ввода), либо откорректировать вручную (если, например, видно, что явно неправильно вбит год). В работе с реальными данными я бы попыталась их сохранить, то есть посмотреть, понять, с чем связана ошибка и скорректировать. Здесь для простоты просто удалим строки.

In [385]:
# Добавляем новый признак. Конечно, период болезни получился не очень реалистичный, но допустим, что здесь болеют долго)

data['days_illness'] = (data.time_checkout - data.time_arrival).dt.days
data.head()

,patient_id,hospital_id,diagnosis,temperature,pulse,time_arrival,time_checkout,recovery,days_illness
0,1,53,insult,34.1,182,2019-02-20,2019-09-07,731,199
1,2,52,arrhythmia,40.5,281,2019-04-22,2019-10-20,1057,181
2,3,52,infarct,37.7,212,2019-04-08,2019-08-01,793,115
3,4,54,insult,40.7,68,2019-07-16,2020-01-27,1698,195
4,5,50,arrhythmia,35.8,162,2019-05-16,2019-11-29,1318,197


In [386]:
# Есть ошибки в данных

data[data['days_illness'] <= 0].head()

,patient_id,hospital_id,diagnosis,temperature,pulse,time_arrival,time_checkout,recovery,days_illness
25,26,54,arrhythmia,38.2,202,2019-09-05,2019-08-18,171,-18
40,41,50,infarct,37.4,256,2019-08-03,2019-07-28,285,-6
67,68,50,insult,34.6,249,2019-08-25,2019-07-02,-99,-54
85,86,51,arrhythmia,39.6,269,2019-09-05,2019-07-26,-36,-41
222,223,51,arrhythmia,36.4,273,2019-07-31,2019-07-22,270,-9


In [387]:
# Удаляем строки

data = data[data['days_illness'] > 0]

data.head()

,patient_id,hospital_id,diagnosis,temperature,pulse,time_arrival,time_checkout,recovery,days_illness
0,1,53,insult,34.1,182,2019-02-20,2019-09-07,731,199
1,2,52,arrhythmia,40.5,281,2019-04-22,2019-10-20,1057,181
2,3,52,infarct,37.7,212,2019-04-08,2019-08-01,793,115
3,4,54,insult,40.7,68,2019-07-16,2020-01-27,1698,195
4,5,50,arrhythmia,35.8,162,2019-05-16,2019-11-29,1318,197


In [388]:
data.shape

(967, 9)

In [389]:
# Уже лучше

get_score(data[['temperature', 'pulse', 'days_illness']], data.recovery)

Weights: [9.37362933 0.06531769 0.67774159]
Bias: 277.07221200731703
Error: 206.36071696154514


Теперь добавим новый признак month_arrival - месяц поступления. Проверим гипотезу о том, что прогноз выздоровления зависит от сезонности. Используем только дату поступления, можно, конечно и дату выписки проверить, но гипотеза будет звучать странно (из серии - "те, кого выписывают в июне, чаще получают повторный инфаркт")

На реальных данных можно было ещё выделить год поступления в больницу, чтобы посмотреть корреляции по годам, но здесь у нас только 2019, поэтому нет смысла.

In [390]:
data['month_arrival'] = (data.time_arrival).dt.month
data.head()

,patient_id,hospital_id,diagnosis,temperature,pulse,time_arrival,time_checkout,recovery,days_illness,month_arrival
0,1,53,insult,34.1,182,2019-02-20,2019-09-07,731,199,2
1,2,52,arrhythmia,40.5,281,2019-04-22,2019-10-20,1057,181,4
2,3,52,infarct,37.7,212,2019-04-08,2019-08-01,793,115,4
3,4,54,insult,40.7,68,2019-07-16,2020-01-27,1698,195,7
4,5,50,arrhythmia,35.8,162,2019-05-16,2019-11-29,1318,197,5


In [391]:
# Ещё лучше

get_score(data[['temperature', 'pulse', 'days_illness', 'month_arrival']], data.recovery)


Weights: [ 9.73059495e+00 -1.21793685e-01  2.85387278e+00  1.35070455e+02]
Bias: -506.63077889819397
Error: 121.55262467310132


Добавим ещё признак, пусть будет коэффициент здоровья coef_health

In [392]:
data['coef_health'] = data.days_illness * data.month_arrival
data.head()

,patient_id,hospital_id,diagnosis,temperature,pulse,time_arrival,time_checkout,recovery,days_illness,month_arrival,coef_health
0,1,53,insult,34.1,182,2019-02-20,2019-09-07,731,199,2,398
1,2,52,arrhythmia,40.5,281,2019-04-22,2019-10-20,1057,181,4,724
2,3,52,infarct,37.7,212,2019-04-08,2019-08-01,793,115,4,460
3,4,54,insult,40.7,68,2019-07-16,2020-01-27,1698,195,7,1365
4,5,50,arrhythmia,35.8,162,2019-05-16,2019-11-29,1318,197,5,985


In [396]:
# Совсем хорошо. Меня смущает то, что Error не равен 0, но это видимо погрешность

get_score(data[['coef_health']], data.recovery)

Weights: [1.]
Bias: 333.0000000000004
Error: 2.0750493131194653e-13
